In [16]:
!pip install --upgrade pip
!pip install --upgrade pydantic
!pip install --upgrade sqlalchemy
!pip install --force-reinstall --upgrade farm-haystack sentence-transformers pandas
!pip install numpy==1.24.4 --force-reinstall

  Using cached pydantic-2.10.3-py3-none-any.whl.metadata (172 kB)
Using cached pydantic-2.10.3-py3-none-any.whl (456 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.19
    Uninstalling pydantic-1.10.19:
      Successfully uninstalled pydantic-1.10.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
farm-haystack 1.26.4 requires pydantic<2, but you have pydantic 2.10.3 which is incompatible.
  Using cached farm_haystack-1.26.4-py3-none-any.whl.metadata (31 kB)
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached boilerpy3-1.0.7-py3-none-any.whl.metadata (5.8 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpx-0.28.0-py3-none-any.whl.metadata (7.1 kB)
  U

  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.20 requires pydantic>=2.7.0, but you have pydantic 1.10.19 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.
langchain 0.3.7 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.19 which is incompatible.


Create a Vector Database of latest news info for fact verification



In [17]:
# any required installations
#!pip install farm-haystack[faiss]
!pip install faiss-gpu
!pip install SQLAlchemy==1.4.48

In [18]:
# verify faiss is installed -- needed for vector database
import faiss
print(faiss.__version__)

1.7.2


In [5]:
# required imports
import requests
from sentence_transformers import SentenceTransformer
from haystack.document_stores.faiss import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever
from haystack.pipelines import DocumentSearchPipeline


In [7]:
# Step 1: Collect trusted news articles from News API
news_api_key = 'ca4a7b633fa443dbab1f09365650e975'
url = f'https://newsapi.org/v2/everything?q=us+technology&sources=bbc-news,reuters,associated-press&from=2024-12-02&to=2024-12-03&apiKey={"ca4a7b633fa443dbab1f09365650e975"}'
response = requests.get(url)
articles = response.json()['articles']

In [8]:
# Step 2: Prepare documents
documents = []
for article in articles:
    content = article['content'] or ''
    documents.append({
        'content': content,
        'meta': {'title': article['title'], 'published_at': article['publishedAt']}
    })

In [9]:
# Step 3: Create a FAISS document store
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", embedding_dim=384, validate_index_sync=False)

In [10]:
# Step 4: Use Sentence-Transformers to create embeddings
retriever = EmbeddingRetriever(document_store=document_store, embedding_model="sentence-transformers/all-MiniLM-L6-v2")
document_store.write_documents(documents)
document_store.update_embeddings(retriever)

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Writing Documents: 10000it [00:00, 177923.78it/s]       


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/1 [00:00<?, ? docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Documents Processed: 10000 docs [00:00, 56747.65 docs/s]


In [11]:
# Step 5: Set up the DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever=retriever)

In [13]:
# Step 6: Run a query
query = "Give me AI news"
results = pipeline.run(query=query)

# Print the results in a user-friendly format
for doc in results['documents']:
    print(f"Document ID: {doc.id}, Content: {doc.content[:200]}")  # Print the first 200 characters of each document

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document ID: 278882a8e92346259553eee5958609de, Content: The BBC has revealed its list of 100 inspiring and influential women from around the world for 2024.
Among them are stranded astronaut Sunita Williams, rape survivor Gisèle Pelicot, actress Sharon S…


Now that we got the vector database to work, we can have it periodically update -- every 6 hours -- for the following topics: news, politics, technology, business

In [14]:
import time
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever
from datetime import datetime, timedelta
import requests

# Initialize FAISS Document Store
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", embedding_dim=768, validate_index_sync=False)

# Initialize Retriever
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/all-mpnet-base-v2",
    use_gpu=True
)

# Function to fetch news from News API
def fetch_news(topic):
    api_key = "ca4a7b633fa443dbab1f09365650e975"  # Replace with your API key
    base_url = "https://newsapi.org/v2/everything"
    sources = "bbc-news,reuters,associated-press"
    today = datetime.utcnow()
    two_days_ago = today - timedelta(days=2)

    params = {
        'q': topic,
        'from': two_days_ago.strftime('%Y-%m-%d'),
        'to': today.strftime('%Y-%m-%d'),
        'language': 'en',
        'sortBy': 'relevancy',
        'apiKey': api_key
    }

    response = requests.get(base_url, params=params)
    articles = response.json().get("articles", [])
    documents = []

    for article in articles:
        content = article['content'] or ''
        documents.append({
            'content': content,
            'meta': {
                'title': article['title'],
                'published_at': article['publishedAt'],
                'source': article['source']['name']
            }
        })
    return documents

# Function to update the vector database
def update_vector_database(topics):
    for topic in topics:
        print(f"Fetching news for topic: {topic}")
        new_documents = fetch_news(topic)

        if new_documents:
            print(f"Updating database with {len(new_documents)} new documents for topic: {topic}")
            document_store.write_documents(new_documents)

    # Update embeddings for new documents
    print("Updating embeddings...")
    document_store.update_embeddings(retriever)
    print("Update complete.")

# Function to periodically update database
def periodic_update(interval_hours, topics):
    while True:
        print(f"Starting update at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        update_vector_database(topics)
        print(f"Next update in {interval_hours} hours.")
        time.sleep(interval_hours * 3600)

# Main execution
if __name__ == "__main__":
    # Configure topics
    available_topics = ["news", "politics", "technology", "business"]
    print("Available topics:", available_topics)
    selected_topics = input("Enter topics to update (comma-separated): ").split(",")
    selected_topics = [topic.strip().lower() for topic in selected_topics if topic.strip().lower() in available_topics]

    if not selected_topics:
        print("No valid topics selected. Exiting...")
    else:
        print(f"Selected topics for updates: {selected_topics}")
        periodic_update(interval_hours=6, topics=selected_topics)

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Updating Embedding:   0%|          | 0/184 [00:00<?, ? docs/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Documents Processed: 10000 docs [00:43, 230.30 docs/s]


Update complete.
Next update in 6 hours.


KeyboardInterrupt: 

Example query:


In [15]:
from haystack.pipelines import DocumentSearchPipeline

# Initialize Document Search Pipeline
pipeline = DocumentSearchPipeline(retriever)

# Query the vector database
query = "What are the latest developments in the tech industry?"
results = pipeline.run(query=query, params={"Retriever": {"top_k": 5}})

# Print results
for result in results["documents"]:
    print(f"Title: {result.meta['title']}")
    print(f"Content: {result.content[:200]}...")
    print(f"Published At: {result.meta['published_at']}")
    print("-" * 50)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Title: The Changing Landscape Of Work And The Evolution Of Cybersecurity Technology
Content: Anand Oswal, SVP &amp; GM of Network Security, Palo Alto Networks.
getty
The world of work is always evolving to keep up with technological advances and increase worker productivity. We saw this wi…...
Published At: 2024-12-02T13:45:00Z
--------------------------------------------------
Title: Smart Manufacturing And Mobility Leaders: Adapt With Tech In 2025
Content: Smart Manufacturing And Mobility Leaders: Adapt With Tech In 2025
Forrester
Asset-intensive industries like manufacturing and transportation quickly feel the pain when energy prices rise, raw mater…...
Published At: 2024-12-02T22:30:14Z
--------------------------------------------------
Title: Intel’s CEO shakeup is an Apple silicon success story in a big way
…...
Published At: 2024-12-02T17:21:00Z
--------------------------------------------------
Title: Balyasny leads the way for multistrategy managers in a roller-coaster Novembe